In [1]:
pip install selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install webdriver_manager

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
pip install WebDriverWait

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that satisfies the requirement WebDriverWait (from versions: none)
ERROR: No matching distribution found for WebDriverWait


In [4]:
from selenium import webdriver

In [5]:
# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

In [6]:
# 크롬드라이버 실행
driver = webdriver.Chrome() 

In [7]:
driver.quit()

In [8]:
#chrome driver 설치 코드
def get_chrome_driver():
    #1. 크롬 옵션 세팅
    chrome_options = webdriver.ChromeOptions()
    #2.driver 생성
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

    return driver

- webdriver : 파이썬과 브라우저를 연결하기 위한 객체
- Service : [최신버전] 어떤 브라우저를 사용하게 할건지 선택
- ChromeDriverManager : [최신버전] Chrome을 사용하기 위한 드라이버 설정

In [9]:
driver = get_chrome_driver()

## 1. 타자 연봉 크롤링

In [12]:
driver = get_chrome_driver()

driver.get("https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx")
driver.implicitly_wait(1)
# "HT","SS","LG","OB",
hitter_name_list = []
hitter_salary_list = []
team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]


for t in team: 
    
    for i in range(1,26):
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)  

        time.sleep(2) # 페이지의 완벽한 로딩을 위한 작업
    
        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)
        ActionChains(driver).click(button).perform()

            
        name = driver.find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblName")
        salary = driver.find_elements(By.ID, "cphContents_cphContents_cphContents_playerProfile_lblSalary")
            
        hitter_name_list.append(name[0].get_attribute('innerHTML'))
        hitter_salary_list.append(salary[0].get_attribute('innerHTML'))
            
            
        driver.back()


In [ ]:
hitter_name_list


In [ ]:
hitter_salary_list


In [ ]:
import pandas as pd

# 데이터 프레임으로 만들기
df_hitter = pd.DataFrame({"선수명":hitter_name_list, "salary":hitter_salary_list})

In [ ]:
df_hitter.shape

In [ ]:
# 데이터 저장
df_hitter.to_csv('./hitter_salary.csv',index=False)

## 2. 투수 연봉 크롤링

In [ ]:
driver = get_chrome_driver()

driver.set_window_size(2000, 1000)
driver.set_window_position(1,0) 
driver.get("https://www.koreabaseball.com/Record/Player/PitcherBasic/BasicOld.aspx")
driver.implicitly_wait(1)
# "HT","SS","LG","OB",
name_list = []
salary_list = []
team = ["HT","SS","LG","OB","SK","KT","HH","LT","NC","WO"]

time.sleep(3) 

for t in team: 
    
    for i in range(1,26):
            
        # 드롭다운
        dropdown = "ctl00$ctl00$ctl00$cphContents$cphContents$cphContents$ddlTeam$ddlTeam"
        select_element = driver.find_element(By.NAME, dropdown)
        select = Select(select_element)
        select.select_by_value(t)
    
        time.sleep(2) 

        # 이름 버튼 눌러서 상세 페이지 열기
        name_css_selector = f'#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr:nth-child({i}) > td:nth-child(2) > a'
        button = driver.find_element(By.CSS_SELECTOR,name_css_selector)

        ActionChains(driver).click(button).perform()
    
        name_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblName"]'
        salary_x = '//*[@id="cphContents_cphContents_cphContents_playerProfile_lblSalary"]'
            
        name = driver.find_elements(By.XPATH,name_x)
        salary = driver.find_elements(By.XPATH, salary_x)

        pitcher_name_list.append(name[0].get_attribute('innerHTML'))
        pitcher_salary_list.append(salary[0].get_attribute('innerHTML'))
            
        driver.back()


In [ ]:
pitcher_salary_list

In [ ]:
pitcher_name_list

In [ ]:
df_salary = pd.DataFrame({"선수명":pitcher_name_list, "salary":pitcher_salary_list})

In [ ]:
# 데이터 저장
df_salary.to_csv('./pitcher_salary.csv',index=False)